In [1]:
import argparse
import random
from pathlib import Path

import dspy
from dspy.evaluate import Evaluate

from ccf.pylib import trait_extractor as te

In [2]:
args = argparse.Namespace(
    examples_json=Path("../data/fna/Asteraceae_examples_2025-03-19c.json"),
    model="ollama_chat/gemma3:27b",
    api_base="http://localhost:11434",
    api_key="",
    train_split=0.1,
    dev_split=0.5,
    test_split=0.4,
    seed=2203673,
)

In [3]:
random.seed(args.seed)

In [4]:
examples = te.read_examples(args.examples_json)
dataset = te.split_examples(examples, args.train_split, args.dev_split)

lm = dspy.LM(args.model, api_base=args.api_base, api_key=args.api_key, cache=False)
dspy.configure(lm=lm)

trait_extractor = dspy.Predict(te.TraitExtractor)

In [5]:
evaluator = Evaluate(
    devset=dataset["dev"],
    metric=te.score_prediction,
    num_threads=1,
    display_progress=True,
    display_table=True, 
    provide_traceback=True,
)

In [6]:
evaluator(trait_extractor, devset=dataset["dev"])

Average Metric: 185.39 / 203 (91.3%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 203/203 [21:38<00:00,  6.39s/it]

2025/03/19 14:39:59 INFO dspy.evaluate.evaluate: Average Metric: 185.39250515228474 / 203 (91.3%)


,text,family,taxon,prompt,example_plant_height,example_leaf_shape,example_leaf_length,example_leaf_width,example_leaf_thickness,example_fruit_type,...,pred_fruit_type,pred_fruit_length,pred_fruit_width,pred_seed_length,pred_seed_width,pred_deciduousness,pred_phenology,pred_habitat,pred_elevation,score_prediction
0,"<b>Perennials,</b> 15–25 cm. <b>Stems</b> erect, puberulent to his...",Asteraceae,Chromolaena frustrata,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...",15–25 cm,elliptic-lanceolate to ovate-lanceolate or deltate-ovate,1.5–4 cm,0.7–2.2 cm,,,...,,,,,,,"Flowering mostly Aug–Nov, sometimes year round","Coastal rock barrens, edges of rockland hammocks, undisturbed sites",0–10 m,✔️ [1.000]
1,"<b>Annuals,</b> 30–150(–400+) cm. <b>Stems</b> erect. <b>Leaves</b...",Asteraceae,Ambrosia trifida,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...",30–150(–400+) cm,rounded-deltate to ovate or elliptic,40–150(–250+) mm,30–70(–200+) mm,,,...,burs,3–5(–7+) mm,,,,,Flowering Jul–Nov,"Disturbed sites, waste places, damp soils",0–1600 m,✔️ [0.857]
2,<b>Plants </b>10–80(–200) cm; nodal spines 0. <b>Leaves</b>: petio...,Asteraceae,Xanthium strumarium,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...",10–80(–200) cm,suborbiculate to ± pentagonal or deltate,4–12(–18+) cm,3–10(–18+) cm,,,...,burs,10–30+ mm,,,,,Flowering Jul–Oct,"Damp or seasonally wet, often alkaline, soils, waste places, margi...",10–2000 m,✔️ [0.837]
3,"<b>Biennials </b>or perennials, 60–150(–200) cm; crown sprouts fro...",Asteraceae,Cirsium virginianum,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...",60–150(–200) cm,linear or linear-elliptic,3–15 cm,,,Cypselae,...,cypsele,4–5 mm,,4–5 mm,,,Flowering summer–fall (Aug–Oct),"Moist savannas, pine barrens, coastal plain bogs",0–150 m,✔️ [0.752]
4,"<b>Annuals,</b> to 60 cm. <b>Stems</b> often spreading or prostrat...",Asteraceae,Centaurea benedicta,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...",60 cm,"tapering to winged petioles, blades lanceolate to oblanceolate",6–25 cm,,,Cypselae,...,cypsele,8–11 mm,,,,,Flowering spring–summer (Apr–Aug),"Roadsides, fields, waste places, sometimes cultivated",0–1300 m,✔️ [0.956]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,"<b>Perennials,</b> 10–60 cm; taproots vertical, thick, fibrous, ca...",Asteraceae,Hypochaeris radicata,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...",10–60 cm,"oblanceolate, lyrate",350 mm,5–30 mm,,Cypselae,...,cypsele,6–10 mm,,,,,Flowering Apr–Nov,"Oak-pine forest, coastal prairie, dunes, waste ground, dry fields,...",0–1600 m,✔️ [0.946]
199,"<b>Perennials,</b> 10–60(–100+) cm. <b>Stems</b> erect. <b>Leaves<...",Asteraceae,Ambrosia psilostachya,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...",10–60(–100+) cm,deltate to lanceolate,20–60(–140) mm,8–35(–50+) mm,,,...,burs,2–3 mm,,,,,Flowering Jul–Oct(–Dec),"Disturbed sites, often wet, alkaline, clay soils",0–2200 m,✔️ [0.857]
200,"<b>Perennials,</b> 50–300 cm (rhizomes often elongate, slender, pl...",Asteraceae,Rudbeckia laciniata,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...",50–300 cm,ovate to lanceolate,15–50 cm,10–25 cm,,Cypselae,...,Cypsele,3–4.5 mm,,,,,,,,✔️ [0.902]
201,<b>Plants </b>50–200 cm; rhizomes short- to long-creeping. <b>Stem...,Asteraceae,Solidago altissima,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...",50–200 cm,tapering to bases; blades oblanceolate,95–150 mm,16–20 mm,,Cypselae,...,cypselae,0.5–1.5 mm,,,,,,,,✔️ [0.779]


91.33

In [7]:
mipro_optimizer = dspy.MIPROv2(
    metric=te.score_prediction,
    auto="medium",
)

optimized_trait_extractor = mipro_optimizer.compile(
    trait_extractor,
    trainset=dataset["train"],
    max_bootstrapped_demos=4,
    requires_permission_to_run=False,
    minibatch=False
)

2025/03/19 14:39:59 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: False
num_candidates: 19
valset size: 32

2025/03/19 14:39:59 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/03/19 14:39:59 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/03/19 14:39:59 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


 44%|████████████████████████████████████████████████████████████████████████████▍                                                                                               | 4/9 [00:27<00:34,  6.93s/it]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 4/19


 44%|████████████████████████████████████████████████████████████████████████████▍                                                                                               | 4/9 [00:31<00:39,  7.90s/it]


...

2025/03/19 16:17:11 INFO dspy.evaluate.evaluate: Average Metric: 30.21530698464162 / 32 (94.4%)
2025/03/19 16:17:11 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 94.42 with parameters ['Predictor 0: Instruction 9', 'Predictor 0: Few-Shot Set 8'].
2025/03/19 16:17:11 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [92.35, 89.81, 93.48, 91.98, 92.26, 92.31, 76.03, 92.33, 73.79, 91.74, 91.89, 93.66, 93.81, 91.71, 93.81, 93.81, 92.34, 76.76, 94.42, 94.42, 93.26, 94.49, 94.42]
2025/03/19 16:17:11 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 94.49
2025/03/19 16:17:11 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/03/19 16:17:11 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 24 / 25 =====



Average Metric: 30.30 / 32 (94.7%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [03:27<00:00,  6.47s/it]

2025/03/19 16:20:38 INFO dspy.evaluate.evaluate: Average Metric: 30.297963169922216 / 32 (94.7%)
2025/03/19 16:20:38 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far! Score: 94.68
2025/03/19 16:20:38 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 94.68 with parameters ['Predictor 0: Instruction 9', 'Predictor 0: Few-Shot Set 3'].
2025/03/19 16:20:38 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [92.35, 89.81, 93.48, 91.98, 92.26, 92.31, 76.03, 92.33, 73.79, 91.74, 91.89, 93.66, 93.81, 91.71, 93.81, 93.81, 92.34, 76.76, 94.42, 94.42, 93.26, 94.49, 94.42, 94.68]
2025/03/19 16:20:38 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 94.68
2025/03/19 16:20:38 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/03/19 16:20:38 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 25 / 25 =====



Average Metric: 30.24 / 32 (94.5%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [03:24<00:00,  6.38s/it]

2025/03/19 16:24:03 INFO dspy.evaluate.evaluate: Average Metric: 30.2375983008548 / 32 (94.5%)
2025/03/19 16:24:03 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 94.49 with parameters ['Predictor 0: Instruction 9', 'Predictor 0: Few-Shot Set 3'].
2025/03/19 16:24:03 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [92.35, 89.81, 93.48, 91.98, 92.26, 92.31, 76.03, 92.33, 73.79, 91.74, 91.89, 93.66, 93.81, 91.71, 93.81, 93.81, 92.34, 76.76, 94.42, 94.42, 93.26, 94.49, 94.42, 94.68, 94.49]
2025/03/19 16:24:03 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 94.68
2025/03/19 16:24:03 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/03/19 16:24:03 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 94.68!


In [8]:
evaluator(optimized_trait_extractor, devset=dataset["dev"])

Average Metric: 193.61 / 203 (95.4%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 203/203 [23:29<00:00,  6.94s/it]

2025/03/19 16:47:32 INFO dspy.evaluate.evaluate: Average Metric: 193.61259868302403 / 203 (95.4%)


,text,family,taxon,prompt,example_plant_height,example_leaf_shape,example_leaf_length,example_leaf_width,example_leaf_thickness,example_fruit_type,...,pred_fruit_type,pred_fruit_length,pred_fruit_width,pred_seed_length,pred_seed_width,pred_deciduousness,pred_phenology,pred_habitat,pred_elevation,score_prediction
0,"<b>Perennials,</b> 15–25 cm. <b>Stems</b> erect, puberulent to his...",Asteraceae,Chromolaena frustrata,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...",15–25 cm,elliptic-lanceolate to ovate-lanceolate or deltate-ovate,1.5–4 cm,0.7–2.2 cm,,,...,,,,,,,"Flowering mostly Aug–Nov, sometimes year round","Coastal rock barrens, edges of rockland hammocks, undisturbed sites",0–10 m,✔️ [0.978]
1,"<b>Annuals,</b> 30–150(–400+) cm. <b>Stems</b> erect. <b>Leaves</b...",Asteraceae,Ambrosia trifida,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...",30–150(–400+) cm,rounded-deltate to ovate or elliptic,40–150(–250+) mm,30–70(–200+) mm,,,...,Burs,3–5(–7+) mm,,,,,Flowering Jul–Nov,"Disturbed sites, waste places, damp soils",0–1600 m,✔️ [0.857]
2,<b>Plants </b>10–80(–200) cm; nodal spines 0. <b>Leaves</b>: petio...,Asteraceae,Xanthium strumarium,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...",10–80(–200) cm,suborbiculate to ± pentagonal or deltate,4–12(–18+) cm,3–10(–18+) cm,,,...,Burs,10–30+ mm,,,,,Flowering Jul–Oct,"Damp or seasonally wet, often alkaline, soils, waste places, margi...",10–2000 m,✔️ [0.857]
3,"<b>Biennials </b>or perennials, 60–150(–200) cm; crown sprouts fro...",Asteraceae,Cirsium virginianum,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...",60–150(–200) cm,linear or linear-elliptic,3–15 cm,,,Cypselae,...,Cypselae,4–5 mm,,,,,Flowering summer–fall (Aug–Oct),"Moist savannas, pine barrens, coastal plain bogs",0–150 m,✔️ [0.929]
4,"<b>Annuals,</b> to 60 cm. <b>Stems</b> often spreading or prostrat...",Asteraceae,Centaurea benedicta,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...",60 cm,"tapering to winged petioles, blades lanceolate to oblanceolate",6–25 cm,,,Cypselae,...,Cypselae,8–11 mm,,,,,spring–summer (Apr–Aug),"Roadsides, fields, waste places",0–1300 m,✔️ [0.939]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,"<b>Perennials,</b> 10–60 cm; taproots vertical, thick, fibrous, ca...",Asteraceae,Hypochaeris radicata,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...",10–60 cm,"oblanceolate, lyrate",350 mm,5–30 mm,,Cypselae,...,Cypselae,6–10 mm,,,,,Apr–Nov,"Oak-pine forest, coastal prairie, dunes, waste ground, dry fields,...",0–1600 m,✔️ [0.931]
199,"<b>Perennials,</b> 10–60(–100+) cm. <b>Stems</b> erect. <b>Leaves<...",Asteraceae,Ambrosia psilostachya,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...",10–60(–100+) cm,deltate to lanceolate,20–60(–140) mm,8–35(–50+) mm,,,...,Burs,2–3 mm,,,,,Flowering Jul–Oct(–Dec),"Disturbed sites, often wet, alkaline, clay soils",0–2200 m,✔️ [0.857]
200,"<b>Perennials,</b> 50–300 cm (rhizomes often elongate, slender, pl...",Asteraceae,Rudbeckia laciniata,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...",50–300 cm,ovate to lanceolate,15–50 cm,10–25 cm,,Cypselae,...,Cypselae,3–4.5 mm,,,,,,,,✔️ [0.988]
201,<b>Plants </b>50–200 cm; rhizomes short- to long-creeping. <b>Stem...,Asteraceae,Solidago altissima,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...",50–200 cm,tapering to bases; blades oblanceolate,95–150 mm,16–20 mm,,Cypselae,...,Cypselae,0.5–1.5 mm,,,,,,,,✔️ [0.963]


95.38

In [9]:
dspy.inspect_history(n=1)





[2025-03-19T16:47:32.281490]

System message:

Your input fields are:
1. `family` (str): The family taxon
2. `taxon` (str): The taxon
3. `text` (str): The species description text
4. `prompt` (str): Extract these traits

Your output fields are:
1. `plant_height` (str): The plant height
2. `leaf_shape` (str): The leaf shape
3. `leaf_length` (str): The leaf length
4. `leaf_width` (str): The leaf width
5. `leaf_thickness` (str): The leaf thickness
6. `fruit_type` (str): The fruit type
7. `fruit_length` (str): The fruit length
8. `fruit_width` (str): The fruit width
9. `seed_length` (str): The seed length
10. `seed_width` (str): The seed witdth
11. `deciduousness` (str): The plant's deciduousness
12. `phenology` (str): The phenology
13. `habitat` (str): The habitat
14. `elevation` (str): The elevation

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## family ## ]]
{family}

[[ ## taxon ## ]]
{taxon}

[[ ## text ## ]]
{text}

[[ ## p

In [10]:
optimized_trait_extractor.save("../data/fna/optimized_trait_extractor_2025-03-19c.json")